In [325]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/hla_ii/bin/imports.ipynb")

Populating the interactive namespace from numpy and matplotlib


In [326]:
classI = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.TCGA.conservative.mut.ClassI.csv',
                     index_col=0)
classII = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.cancer.TCGA.conservative.mut.ClassII.csv',
                     index_col=0)

In [327]:
# remove to go back to all mutations
patient_mutations = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_mutations.cancer.TCGA.conservative.mut.csv',
                                index_col=0)
frequent_mutations = list(patient_mutations.sum().sort_values()[patient_mutations.sum().sort_values() >= 0].index)
print len(frequent_mutations)
classI = classI[frequent_mutations]
classII = classII[frequent_mutations]

1018


In [328]:
# previously 0.5 and 2

tmp1 = pd.DataFrame(sum(classI < 2, axis=1) / len(frequent_mutations) )
tmp1.columns = ['classI']
# was 15??
tmp2 = pd.DataFrame(sum(classII < 10, axis=1) / len(frequent_mutations) )
tmp2.columns = ['classII']
print len(tmp1), len(tmp2)
df = pd.merge(tmp1, tmp2, left_index=True, right_index=True, how='outer')

#df = pd.DataFrame(sum(classI < 2, axis=1) / 1018)
#df.columns = ['classI']

5942 5942


In [329]:
df.shape

(5942, 2)

In [330]:
df.head()

,classI,classII
TCGA-02-0003,0.475442,0.330059
TCGA-02-0033,0.524558,0.327112
TCGA-02-0047,0.441061,0.348723
TCGA-02-0055,0.523576,0.328094
TCGA-02-2466,0.584479,0.318271


Covariates

In [331]:
patient_tissues = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/clinical/tissue_type/patient_tissues.csv', index_col=0)
patient_tissues.index = patient_tissues.Sample
df['Tissue'] = patient_tissues.ix[df.index].Tissue 

In [332]:
age = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/clinical/age_at_diagnosis.csv', index_col=0)
df['Age'] = age.ix[df.index].age

In [333]:
ancestry = pd.read_csv('/cellar/users/ramarty/Data/hla/git_data/data/clinical/ancestory.csv', index_col=0)
df['Race'] = ancestry.ix[df.index].race

In [334]:
ancestry.head()

,race
TCGA-02-0001,WHITE
TCGA-02-0003,WHITE
TCGA-02-0004,WHITE
TCGA-02-0006,WHITE
TCGA-02-0007,WHITE


In [335]:
homozygosity = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/mhc_diversity/homozygous_counts.csv',
                           index_col=0)
df['homozygous_classI'] = homozygosity.ix[df.index].homozygous_classI
df['homozygous_classII'] = homozygosity.ix[df.index].homozygous_classII
df['homozygous_total'] = homozygosity.ix[df.index].homozygous_total

In [336]:
len(df)

5942

Add passenger space

In [337]:
passengerI = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.passenger.TCGA.conservative.mut.ClassI.csv',
                     index_col=0)
passengerII = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/presentation/clean_matrices/patient_affinities.passenger.TCGA.conservative.mut.ClassII.csv',
                     index_col=0)
df_passI = pd.DataFrame(mean(passengerI < 2, axis=1))
df_passI.columns = ['passenger_coverageI']
df_passII = pd.DataFrame(mean(passengerII < 10, axis=1))
df_passII.columns = ['passenger_coverageII']
df_pass = pd.merge(df_passI, df_passII, left_index=True, right_index=True)

In [338]:
len(df_pass)

5942

In [339]:
df = pd.merge(df, df_pass, left_index=True, right_index=True, how='inner')

In [340]:
len(df)

5942

Patients to drop

In [341]:
to_drop = pd.read_csv('/cellar/users/ramarty/Data/hla_ii/clinical/Lynch.MSIH.labels', sep='\t', header=None)
to_drop.columns = ['patient', 'disease']

In [342]:
patients_to_drop = list(to_drop[to_drop.disease == 'LYNCH'].patient)

In [343]:
df = df.ix[[x for x in df.index if x not in patients_to_drop]]

Splitting into coverage groups

In [344]:
def assign_categoryI(x):
    if x < df.classI.dropna().quantile(L):
        return 'low'
    elif x > df.classI.dropna().quantile(H):
        return 'high'
    else:
        return 'other'
def assign_categoryII(x):
    if x < df.classII.dropna().quantile(L):
        return 'low'
    elif x > df.classII.dropna().quantile(H):
        return 'high'
    else:
        return 'other'

In [345]:
L, H = 0.25, 0.75
df['classI_cat'] = df.classI.apply(assign_categoryI)
df['classII_cat'] = df.classII.apply(assign_categoryII)

In [346]:
df.classI.dropna().quantile(L)

0.48231827111984282

In [347]:
assign_categoryI(0.169941)

'low'

In [348]:
def assign_categoryI(x):
    if x == 0:
        return 'het'
    elif x >= 2:
        return 'homo'
    else:
        return 'other'
def assign_categoryII(x):
    if x <= 2:
        return 'het'
    else:
        return 'homo'

In [349]:
df['classI_het'] = df.homozygous_classI.apply(assign_categoryI)
df['classII_het'] = df.homozygous_classII.apply(assign_categoryII)

Output

In [350]:
df.dropna().to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/mhc_diversity/dataframe.nonan.csv')

In [351]:
df.to_csv('/cellar/users/ramarty/Data/hla_ii/generated_data/mhc_diversity/dataframe.csv')

In [323]:
print df.dropna().shape, df.shape

(4573, 14) (5786, 14)


In [296]:
df.homozygous_classII.value_counts()

1.0    2324
0.0    1662
2.0    1280
3.0     286
4.0     160
5.0      74
Name: homozygous_classII, dtype: int64

In [42]:
df.homozygous_classI.value_counts()

0.0    5895
1.0    1523
2.0     395
3.0     221
Name: homozygous_classI, dtype: int64